LnagChain을 활용한 문서 요약

In [ ]:
!pip install langchain transformers


In [ ]:
!pip install langchain_community

In [11]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import pipeline

from langchain.llms import HuggingFacePipeline

In [4]:
# 1. 예제 텍스트
text_to_summarize = """
Artificial intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think
like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with
a human mind such as learning and problem-solving. The ideal characteristic of artificial intelligence is its ability
to rationalize and take actions that have the best chance of achieving a specific goal. A subset of AI is machine
learning, which refers to the concept that computer programs can automatically learn from and adapt to new data without
being assisted by humans. Deep learning techniques enable this automatic learning through the absorption of huge amounts
of unstructured data such as text, images, or video.
"""


In [5]:
# 2. 텍스트 요약을 위한 프롬프트 템플릿 생성
# 요약 생성을 위한 프롬프트 정의
prompt_template = """
Summarize the following text in one concise paragraph:

{text}
"""

In [6]:
# 3. 요약기 모델 설정 (Hugging Face 모델)
# pipeline : Hugging Face에서 제공하는 사전 학습된 요약 모델 호출
# summarization 태스크를 위한 파이프라인 설정

summarizer = pipeline("summarization")



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [12]:
# 4. Chain 생성
# LLMChain 사용,  프롬프트 템플릿과 요약 모델 결합

# prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
# chain = LLMChain(llm=summarizer, prompt=prompt)

llm = HuggingFacePipeline(pipeline=summarizer)

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = LLMChain(llm=llm, prompt=prompt)  # Use the wrapped llm


In [13]:
# 5. 요약 실행
# summarizer 사용 텍스트 요약
# 최대 100 단어, 최소 30 단어의 길이 요약 결과 생성
summary = summarizer(text_to_summarize, max_length=100, min_length=30, do_sample=False)

In [14]:
# 결과 출력
print("Original Text:\n", text_to_summarize)
print("\nSummarized Text:\n", summary[0]['summary_text'])


Original Text:
 
Artificial intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think
like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with
a human mind such as learning and problem-solving. The ideal characteristic of artificial intelligence is its ability
to rationalize and take actions that have the best chance of achieving a specific goal. A subset of AI is machine
learning, which refers to the concept that computer programs can automatically learn from and adapt to new data without
being assisted by humans. Deep learning techniques enable this automatic learning through the absorption of huge amounts
of unstructured data such as text, images, or video.


Summarized Text:
  Artificial intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions . The ideal characteristic of artificial intelli

한국어 요약

In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

In [ ]:
# 1. 한국어 처리를 위한 Kobart 모델, 토크나이저
# gogamza/kobart-summarization 모델 사용, 한국어 문서 요약

model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')

In [ ]:
# 2. 예제 텍스트 (한국어)
text_to_summarize = """
인공지능(AI)은 기계가 인간의 지능적 행동을 모방할 수 있게 하는 기술입니다. AI는 데이터 분석, 학습 및 문제 해결 능력을 통해 복잡한 문제를 해결하는 데 사용됩니다.
기계 학습은 AI의 하위 분야로, 데이터로부터 학습하고 자동으로 개선되는 알고리즘을 개발하는 것에 중점을 둡니다.
딥러닝은 대규모의 데이터를 처리하고 분석하는 기술로, 인공 신경망을 사용하여 데이터를 모델링합니다.
"""

In [ ]:
# 3. 토큰화
inputs = tokenizer.encode(text_to_summarize, return_tensors='pt', max_length=512, truncation=True)

In [ ]:
# 4. 요약 생성
# model.generate() 사용, 요약 생성
# length_penalty 모델이 너무 짧거나 너무 긴 텍스트를 생성하지 않도록 제어
   # 1.0: 페널티 적용 X -> 텍스트 길이에 대한 제약 없이 텍스트 생성
   # > 1.0: 긴 텍스트에 페널티 부여 -> 값이 클수록 모델이 짧은 텍스트 선호
   # < 1.0: 짧은 텍스트에 페널티 부여 -> 값이 작을수록 더 긴 텍스트 생성

#  num_beams : 텍스트 생성 과정에서 탐색할 경로의 수  지정 -> 생성되는 텍스트의 다양성과 품질을 제어
   # 빔의 수가 많을수록 더 다양한 선택지 고려
   # 1: 빔 서치 사용 X, 그리디 서치(Greedy Search) 사용(가장 가능성이 높은 단어 선택)
   # > 2 : 여러 경로 동시에 탐색. 값이 클수록 더 많은 경로 탐색 but 계산 비용 UP
   # ex, num_beams=5: 다섯 개의 경로 동시 탐색

summary_ids = model.generate(inputs, max_length=128, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
# 5. 결과 출력
print("Original Text:\n", text_to_summarize)
print("\nSummarized Text:\n", summary)


Original Text:
 
인공지능(AI)은 기계가 인간의 지능적 행동을 모방할 수 있게 하는 기술입니다. AI는 데이터 분석, 학습 및 문제 해결 능력을 통해 복잡한 문제를 해결하는 데 사용됩니다. 
기계 학습은 AI의 하위 분야로, 데이터로부터 학습하고 자동으로 개선되는 알고리즘을 개발하는 것에 중점을 둡니다. 
딥러닝은 대규모의 데이터를 처리하고 분석하는 기술로, 인공 신경망을 사용하여 데이터를 모델링합니다.


Summarized Text:
 인공지능인공지능(AI)은 기계가 인간의 지능적 행동을 모방할 수 있게 하는 기술인만큼인공지능(AI)은 기계가 인간의 지능적 행동을 모방할 수 있게 하는 기술인만큼인공지능(AI)은 기계가 인간의 지능적 행동을 모방할 수 있게 하는 기술입니다.
